# <center>Data Mining project: Discover and describe areas of interest<br> and events from geo-located data</center>

## #1: Import Dataset and Libraries

## Environment

In [1]:
! python -m venv dataMiningEnv
#Activate windows
! dataMiningEnv\Scripts\activate.bat


Actual environment location may have moved due to redirects, links or junctions.
  Requested location: "h:\IF4\fouilleD\projet\dataMiningEnv\Scripts\python.exe"
  Actual location:    "\\ps-home.insa-lyon.fr\users\home\dlarrazmar\IF4\fouilleD\projet\dataMiningEnv\Scripts\python.exe"


## Libraries

- ### Installs

In [2]:
# installation of required libraries and dependencies
# numeric calculations
! pip install numpy==1.26.0 
# data frames 
! pip install pandas==2.1.1 
# machine learning algorithms 
! pip install scikit-learn==1.5.1 
! pip install scipy==1.12.0
# plotting 
! pip install plotly==5.24.1 
! pip install matplotlib==3.8.0 
! pip install seaborn==0.13.2 
! pip install plotly-express==0.4.1 
! pip install chart-studio==1.1.0 
# web app library 
! pip install streamlit==1.37.1 
# association rules
! pip install mlxtend==0.23.3
# Language processing
! pip install nltk
! python -m nltk.downloader popular # popular functions


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
^C


- ### Imports

In [3]:
import pandas as pd
#import nltk



## Data

In [13]:
# Charger les données avec `low_memory=False` pour éviter les avertissements
data = pd.read_csv("data/flickr_data2.csv", sep=",", low_memory=False)

# Supprimer les espaces supplémentaires dans les noms des colonnes
data.columns = data.columns.str.strip()

# Convertir les colonnes contenant des types mixtes (object) en numériques
# Liste des colonnes de type temps
time_columns = [
    'date_taken_minute', 'date_taken_hour', 'date_taken_day',
    'date_taken_month', 'date_taken_year',
    'date_upload_minute', 'date_upload_hour', 'date_upload_day',
    'date_upload_month', 'date_upload_year'
]

# Convertir chaque colonne en int64
for col in time_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0).astype('int64')


data.head(5)



,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year,date_upload_minute,date_upload_hour,date_upload_day,date_upload_month,date_upload_year,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,11,15,28,2,2010,23,20,28,2,2010,NaN,NaN,NaN
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,51,17,28,2,2010,52,17,28,2,2010,NaN,NaN,NaN
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,29,17,28,2,2010,33,17,28,2,2010,NaN,NaN,NaN
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,15,20,28,1,2010,38,12,28,2,2010,NaN,NaN,NaN
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,10,20,28,1,2010,38,12,28,2,2010,NaN,NaN,NaN


## #2: Exploratory Data Analysis (Cleaning)

In [6]:
def parse_conclusion(pre_data, parsed_data):
    l = len(pre_data)-len(parsed_data)
    print(f"<Lines parsed: {l} - {round(100*l/len(data),3)}% of original data>")

## Duplicates

In [7]:
# Save parsed lines
data[data['id'].duplicated(keep='first')].sort_values("id").to_csv("data/parsed_lines/duplicatedId.csv", index = False)

# Parse
unduplicated_id_data = data[~data['id'].duplicated(keep='first')]
parse_conclusion(data, unduplicated_id_data)

<Lines parsed: 252143 - 60.0% of original data>


## Corrupted Data and Shifted columns due to ";" in title
Examples:
- https://flickr.com/photos/29713277@N02/6674970791
- https://flickr.com/photos/53987060@N02/33105832478

The first one clearly shows that if the title contains ";", the data inside of the csv is shuffled.<br>
The second example has a sneaky ";" within the smily face at the end of the title.

In [14]:
# Save parsed lines
corrupted_data = unduplicated_id_data[unduplicated_id_data["Unnamed: 16"].notnull() | unduplicated_id_data["Unnamed: 17"].notnull() | unduplicated_id_data["Unnamed: 18"].notnull()]
corrupted_data.to_csv("data/parsed_lines/corrupted_data.csv", index = False)

# Parse
uncorrupted_data = unduplicated_id_data[~(unduplicated_id_data["Unnamed: 16"].notnull() | unduplicated_id_data["Unnamed: 17"].notnull() | unduplicated_id_data["Unnamed: 18"].notnull())]

parse_conclusion(unduplicated_id_data, uncorrupted_data)

<Lines parsed: 47 - 0.011% of original data>


- ### Unused variables 

In [15]:
# Drop unused columns
uncorrupted_data = uncorrupted_data.drop(columns=["Unnamed: 16", "Unnamed: 17", "Unnamed: 18", "date_upload_minute", "date_upload_hour", "date_upload_day", "date_upload_month", "date_upload_year"])
uncorrupted_data.head(3)

,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,11,15,28,2,2010
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,51,17,28,2,2010
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,29,17,28,2,2010


## Pas chez eux

## Corrupt coords and dates

## (POST PROCESS !!!!) Same user, space and approximated time => many captures taken on site

In [17]:
# Save parsed lines PAS CA!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
multipic_data = uncorrupted_data[uncorrupted_data[['user', 'lat', 'long', 'date_taken_day', 'date_taken_month', 'date_taken_year']].duplicated(keep='first')].sort_values("id")
multipic_data.to_csv("data/parsed_lines/multipic.csv", index = False)

# Parse
onlyonepic_data = uncorrupted_data[~uncorrupted_data[['user', 'lat', 'long', 'tags', 'date_taken_day', 'date_taken_month', 'date_taken_year']].duplicated(keep='first')]

parse_conclusion(uncorrupted_data, onlyonepic_data)

<Lines parsed: 87113 - 20.729% of original data>
